In [0]:
# COMMAND ----------
# Install the Azure Event Hubs library (This only needs to run once per cluster start)
%pip install azure-eventhub

# COMMAND ----------
import time
import json
import random
from azure.eventhub import EventHubProducerClient, EventData

# --- CONFIGURATION ---
CONNECTION_STR = "Endpoint=sb://eh-retail-stream-tithir.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=zh4CpF2GknXjtj7o/JRpqJE17mPE/RGTV+AEhJzqptE="
EVENT_HUB_NAME = "telemetry_stream"

def generate_sensor_data():
    devices = ['sensor-A', 'sensor-B', 'sensor-C', 'sensor-D']
    return {
        "device_id": random.choice(devices),
        "temperature": round(random.uniform(20.0, 35.0), 2),
        "humidity": round(random.uniform(40.0, 80.0), 2),
        "status": random.choice(['OK', 'OK', 'OK', 'WARNING']),
        "timestamp": time.time()
    }

# --- MAIN LOOP ---
def run_generator():
    print(f"🚀 Connecting to {EVENT_HUB_NAME}...")
    producer = EventHubProducerClient.from_connection_string(
        conn_str=CONNECTION_STR, 
        eventhub_name=EVENT_HUB_NAME
    )
    
    with producer:
        print("✅ Connected! Sending stream... (Press 'Stop' or 'Cancel' to end)")
        while True:
            batch = producer.create_batch()
            
            # Create a batch of 5 fake events
            for _ in range(5):
                data = generate_sensor_data()
                batch.add(EventData(json.dumps(data)))
            
            producer.send_batch(batch)
            print(f"Sent 5 events at {time.strftime('%H:%M:%S')}")
            time.sleep(2) # Wait 2 seconds

# Run the generator
run_generator()

In [0]:
%sql
-- Is the data landing?
SELECT * FROM azure_databricks_personal.default.iot_telemetry_realtime 
ORDER BY timestamp DESC 
LIMIT 10